In [1]:
import os
import json
import pandas as pd
import numpy as np
import statistics
from helpers import *
import re
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv(r'Dataset_Big.csv')
df.head()

C:\Users\swak\AppData\Local\Temp\ipykernel_18416\52725572.py:1: DtypeWarning: Columns (228,302,307,309,316) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'Dataset_Big.csv')


,IMPORT_TABLE_size,strings_printabledist_31,strings_printabledist_88,.tls_vsize,subsystem,minor_image_version,RESOURCE_TABLE_virtual_address,.idata_vsize,.textbss_size,Dbgcore.dll,...,.idat_size,Libcrypto_num_funcs,mbedTLS,.txt_entropy,.UPX_props_len,CryptoAPI_num_funcs,.idat_props_len,.idat_vsize,.txt_size,.txt_vsize
0,200,858,965,8,WINDOWS_GUI,0,352256,2370,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,420,48,170,8,WINDOWS_GUI,20512,483328,8192,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,216,116,152,8,WINDOWS_GUI,0,90112,2242,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2370,613,653,8,WINDOWS_GUI,0,65536,2384,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,140,288,323,8,WINDOWS_GUI,0,49152,2384,0,True,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Save our feature list
feature_columns = list(df.columns)
feature_columns.pop(feature_columns.index('label'))
print(f"total number of Features: {len(feature_columns)}")
with open(os.path.join(os.getcwd(), 'assets', 'features.pkl'), 'wb') as f:
    pickle.dump(feature_columns, f)

total number of Features: 323


# Discard -1 rows

In [4]:
df = df[df['label'] != -1]

# Encode categorical columns

In [5]:
from sklearn.preprocessing import LabelEncoder


with open(os.path.join(os.getcwd(), 'assets', 'suspicious_imports.txt'), 'r') as f:
        sus_imports = f.readlines()
sus_imports = [re.sub(r'\n', '', i) for i in sus_imports]
boolean_columns = sus_imports + []
categorical_columns = ["subsystem", "magic", "machine"]

array_of_Label_Encoders = []
for col in categorical_columns:
    new_LE = LabelEncoder().fit(df[col])
    df[col] = new_LE.transform(df[col])
    array_of_Label_Encoders.append(new_LE)


for col in df.columns:
        if col in boolean_columns:
            df[col] = df[col].astype(bool)
            df[col].fillna(False)
            continue

with open(os.path.join(os.getcwd(), 'models', 'enc.pkl'), 'wb') as f:
    pickle.dump(array_of_Label_Encoders, f)



with open(os.path.join(os.getcwd(),'models','enc.pkl'), 'rb') as f:
        array_of_Label_Encoders = pickle.load(f)


# Random Forest

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

df_train_2 = df.copy()


feature_columns = list(df_train_2.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)


x_train, x_test, y_train, y_test = train_test_split(df_train_2[feature_columns], df_train_2['label'], test_size=0.3, shuffle=True)

rf_model = RandomForestClassifier().fit(x_train, y_train)

y_pred = rf_model.predict(x_test)
print(classification_report(y_test, y_pred))


with open(os.path.join(os.getcwd(), 'models', 'rf.pkl'), 'wb') as f:
    pickle.dump(rf_model, f)


              precision    recall  f1-score   support

           0       0.95      0.97      0.96     90046
           1       0.97      0.95      0.96     89954

    accuracy                           0.96    180000
   macro avg       0.96      0.96      0.96    180000
weighted avg       0.96      0.96      0.96    180000



# NN 1

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class MyNet(nn.Module):
    def __init__(self, num_features = 120):
        super(MyNet, self).__init__()
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, 512)
        self.batch_norm2 = nn.BatchNorm1d(512)
        self.dense2 = nn.Linear(512, 128)
        self.batch_norm3 = nn.BatchNorm1d(128)
        self.dense3 = nn.Linear(128, 8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.batch_norm1(x.float())
        x = torch.tanh(self.dense1(x))
        x = self.batch_norm2(x.float())
        x = torch.tanh(self.dense2(x))
        x = self.batch_norm3(x.float())
        x = torch.tanh(self.dense3(x))
        x = self.softmax(x)
        return x

# Create an instance of the network
net = MyNet()

# Print the network architecture
print(net)




# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe_train, dataframe_labels):
        self.data = dataframe_train
        self.labels = dataframe_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index, :]
        y = self.labels[index]
        return x, y



# Define the testing function
def test_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    device = next(model.parameters()).device  # Get the device of the model
    
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            
            # Forward pass
            outputs = model(inputs)
            # Get the predicted labels
            #preds = torch.round(torch.sigmoid(outputs))
            _, preds = torch.max(outputs, 1)  # Get the predicted labels
            
            # Collect the labels and predictions
            all_labels += list(labels.numpy().reshape((-1,1)))
            all_preds += list(preds.numpy().reshape((-1,1)))
    
    return np.asarray(all_labels), np.asarray(all_preds)



MyNet(
  (batch_norm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=120, out_features=512, bias=True)
  (batch_norm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense2): Linear(in_features=512, out_features=128, bias=True)
  (batch_norm3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense3): Linear(in_features=128, out_features=8, bias=True)
  (softmax): Softmax(dim=1)
)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64))
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64))
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)#, weight_decay=0.0001)

# Training loop
for epoch in range(130):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')

# Save the trained model
#torch.save(net.state_dict(), 'trained_model.pt')

with open(os.path.join(os.getcwd(), 'models', 'NN1.pkl'), 'wb') as f:
     pickle.dump(net, f)
    

test_dataset = MyDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net, test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))

Epoch 1: Loss = 1.9038, Train Accuracy = 0.6314, Val Accuracy = 0.6866
Epoch 2: Loss = 1.8401, Train Accuracy = 0.6937, Val Accuracy = 0.6981
Epoch 3: Loss = 1.8305, Train Accuracy = 0.6972, Val Accuracy = 0.7006
Epoch 4: Loss = 1.8249, Train Accuracy = 0.7056, Val Accuracy = 0.7170
Epoch 5: Loss = 1.8194, Train Accuracy = 0.7180, Val Accuracy = 0.7240
Epoch 6: Loss = 1.8141, Train Accuracy = 0.7319, Val Accuracy = 0.7353
Epoch 7: Loss = 1.8091, Train Accuracy = 0.7431, Val Accuracy = 0.7479
Epoch 8: Loss = 1.8051, Train Accuracy = 0.7525, Val Accuracy = 0.7580
Epoch 9: Loss = 1.7996, Train Accuracy = 0.7696, Val Accuracy = 0.7833
Epoch 10: Loss = 1.7863, Train Accuracy = 0.8025, Val Accuracy = 0.8096
Epoch 11: Loss = 1.7779, Train Accuracy = 0.8216, Val Accuracy = 0.8282
Epoch 12: Loss = 1.7717, Train Accuracy = 0.8345, Val Accuracy = 0.8365
Epoch 13: Loss = 1.7692, Train Accuracy = 0.8399, Val Accuracy = 0.8384
Epoch 14: Loss = 1.7675, Train Accuracy = 0.8442, Val Accuracy = 0.8427
E

## NN2
### same as base NN model, but without any batch normalization

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class MyNet(nn.Module):
    def __init__(self, num_features = 120):
        super(MyNet, self).__init__()

        self.dense1 = nn.Linear(num_features, 512)
        self.dense2 = nn.Linear(512, 128)
        self.dense3 = nn.Linear(128, 8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = torch.tanh(self.dense1(x))
        x = torch.tanh(self.dense2(x))
        x = torch.tanh(self.dense3(x))
        x = self.softmax(x)
        return x

# Create an instance of the network
net = MyNet()

# Print the network architecture
print(net)

# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe_train, dataframe_labels):
        self.data = dataframe_train
        self.labels = dataframe_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index, :]
        y = self.labels[index]
        return x, y

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64)).type(torch.float)
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64)).type(torch.float)
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)#, weight_decay=0.0001)

# Training loop
for epoch in range(130):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')


with open(os.path.join(os.getcwd(), 'models', 'NN2.pkl'), 'wb') as f:
     pickle.dump(net, f)



test_dataset = MyDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net, test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))

# NN3
### same as base model, but with Leaky Relu instead of the tanh activation functions

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class MyNet(nn.Module):
    def __init__(self, num_features = 120):
        super(MyNet, self).__init__()
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, 512)
        self.lrelu1 = nn.LeakyReLU()
        self.batch_norm2 = nn.BatchNorm1d(512)
        self.dense2 = nn.Linear(512, 128)
        self.lrelu2 = nn.LeakyReLU()
        self.batch_norm3 = nn.BatchNorm1d(128)
        self.dense3 = nn.Linear(128, 8)
        self.lrelu3 = nn.LeakyReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.batch_norm1(x.float())
        x = self.lrelu1(self.dense1(x))
        x = self.batch_norm2(x.float())
        x = self.lrelu2(self.dense2(x))
        x = self.batch_norm3(x.float())
        x = self.lrelu3(self.dense3(x))
        x = self.softmax(x)
        return x

# Create an instance of the network
net = MyNet()

# Print the network architecture
print(net)




# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe_train, dataframe_labels):
        self.data = dataframe_train
        self.labels = dataframe_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index, :]
        y = self.labels[index]
        return x, y

MyNet(
  (batch_norm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=120, out_features=512, bias=True)
  (lrelu1): LeakyReLU(negative_slope=0.01)
  (batch_norm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense2): Linear(in_features=512, out_features=128, bias=True)
  (lrelu2): LeakyReLU(negative_slope=0.01)
  (batch_norm3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense3): Linear(in_features=128, out_features=8, bias=True)
  (lrelu3): LeakyReLU(negative_slope=0.01)
  (softmax): Softmax(dim=1)
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64))
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64))
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)#, weight_decay=0.0001)

# Training loop
for epoch in range(130):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')

# Save the trained model
#torch.save(net.state_dict(), 'trained_model.pt')

with open(os.path.join(os.getcwd(), 'models', 'NN3.pkl'), 'wb') as f:
     pickle.dump(net, f)


test_dataset = MyDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net, test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))

Epoch 1: Loss = 1.6818, Train Accuracy = 0.6489, Val Accuracy = 0.7792
Epoch 2: Loss = 1.4998, Train Accuracy = 0.7909, Val Accuracy = 0.8000
Epoch 3: Loss = 1.4754, Train Accuracy = 0.8058, Val Accuracy = 0.8087
Epoch 4: Loss = 1.4629, Train Accuracy = 0.8158, Val Accuracy = 0.8168
Epoch 5: Loss = 1.4547, Train Accuracy = 0.8223, Val Accuracy = 0.8196
Epoch 6: Loss = 1.4471, Train Accuracy = 0.8269, Val Accuracy = 0.8232
Epoch 7: Loss = 1.4420, Train Accuracy = 0.8305, Val Accuracy = 0.8238
Epoch 8: Loss = 1.4382, Train Accuracy = 0.8336, Val Accuracy = 0.8266
Epoch 9: Loss = 1.4364, Train Accuracy = 0.8350, Val Accuracy = 0.8292
Epoch 10: Loss = 1.4334, Train Accuracy = 0.8376, Val Accuracy = 0.8319
Epoch 11: Loss = 1.4301, Train Accuracy = 0.8407, Val Accuracy = 0.8339
Epoch 12: Loss = 1.4284, Train Accuracy = 0.8423, Val Accuracy = 0.8375
Epoch 13: Loss = 1.4262, Train Accuracy = 0.8443, Val Accuracy = 0.8378
Epoch 14: Loss = 1.4246, Train Accuracy = 0.8461, Val Accuracy = 0.8406
E

KeyboardInterrupt: 

# NN 4
### same base model but with an extra dense layer and weight decay = 0.000001

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class MyNet(nn.Module):
    def __init__(self, num_features = 120):
        super(MyNet, self).__init__()
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, 512)
        self.batch_norm2 = nn.BatchNorm1d(512)
        self.dense2 = nn.Linear(512, 512)
        self.batch_norm3 = nn.BatchNorm1d(512)
        self.dense3 = nn.Linear(512, 128)
        self.batch_norm4 = nn.BatchNorm1d(128)
        self.dense4 = nn.Linear(128, 8)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.batch_norm1(x.float())
        x = torch.tanh(self.dense1(x))
        x = self.batch_norm2(x.float())
        x = torch.tanh(self.dense2(x))
        x = self.batch_norm3(x.float())
        x = torch.tanh(self.dense3(x))
        x = self.batch_norm4(x.float())
        x = torch.tanh(self.dense4(x))
        x = self.softmax(x)
        return x

# Create an instance of the network
net = MyNet()

# Print the network architecture
print(net)




# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe_train, dataframe_labels):
        self.data = dataframe_train
        self.labels = dataframe_labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index, :]
        y = self.labels[index]
        return x, y

MyNet(
  (batch_norm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=120, out_features=512, bias=True)
  (batch_norm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense2): Linear(in_features=512, out_features=512, bias=True)
  (batch_norm3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense3): Linear(in_features=512, out_features=128, bias=True)
  (batch_norm4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense4): Linear(in_features=128, out_features=8, bias=True)
  (softmax): Softmax(dim=1)
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64))
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64))
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.000001)

# Training loop
for epoch in range(130):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')

# Save the trained model
#torch.save(net.state_dict(), 'trained_model.pt')

with open(os.path.join(os.getcwd(), 'models', 'NN4.pkl'), 'wb') as f:
     pickle.dump(net, f)


test_dataset = MyDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Test the model
true_labels, predicted_labels = test_model(net, test_dataloader)

# Import the necessary libraries for classification report
from sklearn.metrics import classification_report

# Print the classification report
target_names = ['class_0', 'class_1']  # Replace with appropriate class names
print(classification_report(true_labels, predicted_labels, labels=np.unique(true_labels)))

Epoch 1: Loss = 1.8411, Train Accuracy = 0.6763, Val Accuracy = 0.7346
Epoch 2: Loss = 1.7978, Train Accuracy = 0.7681, Val Accuracy = 0.7936
Epoch 3: Loss = 1.7753, Train Accuracy = 0.8268, Val Accuracy = 0.8448
Epoch 4: Loss = 1.7620, Train Accuracy = 0.8612, Val Accuracy = 0.8633
Epoch 5: Loss = 1.7537, Train Accuracy = 0.8795, Val Accuracy = 0.8751
Epoch 6: Loss = 1.7498, Train Accuracy = 0.8888, Val Accuracy = 0.8839
Epoch 7: Loss = 1.7464, Train Accuracy = 0.8964, Val Accuracy = 0.8886
Epoch 8: Loss = 1.7440, Train Accuracy = 0.9023, Val Accuracy = 0.8943
Epoch 9: Loss = 1.7421, Train Accuracy = 0.9071, Val Accuracy = 0.8924
Epoch 10: Loss = 1.7406, Train Accuracy = 0.9109, Val Accuracy = 0.9022
Epoch 11: Loss = 1.7396, Train Accuracy = 0.9132, Val Accuracy = 0.9027
Epoch 12: Loss = 1.7382, Train Accuracy = 0.9167, Val Accuracy = 0.9062
Epoch 13: Loss = 1.7373, Train Accuracy = 0.9189, Val Accuracy = 0.9061
Epoch 14: Loss = 1.7363, Train Accuracy = 0.9215, Val Accuracy = 0.9079
E

KeyboardInterrupt: 

# NN 5
### same as NN 4 but with bigger learning rate, and with weight decay

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"


df_train_3 = df.copy()


feature_columns = list(df_train_3.columns)
feature_columns.pop(feature_columns.index("label"))

if(re.findall('Unnamed', feature_columns[0], re.IGNORECASE)):
    feature_columns.pop(0)
    
num_features = len(feature_columns)


x_train, x_test, y_train, y_test = train_test_split(df_train_3.drop(columns=['label']), df_train_3['label'], test_size=0.3, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)


# Split the data into features and labels
X = torch.from_numpy(np.asarray(x_train, dtype=np.int64))
y = torch.from_numpy(np.asarray(y_train, dtype=bool)).type(torch.int64)

# Split the data into features and labels
X_val = torch.from_numpy(np.asarray(x_val, dtype=np.int64))
y_val = torch.from_numpy(np.asarray(y_val, dtype=bool)).type(torch.int64)

# Create instances of the dataset and data loader
dataset = MyDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

val_dataset = MyDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)



# Create an instance of the network
net = MyNet(num_features).to(device=device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device=device)
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.000001)

# Training loop
for epoch in range(130):  # Replace 10 with the desired number of epochs
    running_loss = 0.0
    running_corrects = 0  # Counter for correct predictions
    
    for inputs, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs.to(device=device))
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
        
        # Calculate the predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get the predicted labels
        corrects = torch.sum(preds == labels)  # Count the number of correct predictions
        running_corrects += corrects.item()
    
    # Calculate the running accuracy on the training set
    train_accuracy = running_corrects / len(dataset)
     
    # Calculate the validation accuracy
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = net(inputs)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()
            val_total += labels.size(0)
    val_accuracy = val_corrects / val_total
    
    # Print the average loss and accuracy for the epoch
    print(f'Epoch {epoch+1}: Loss = {running_loss/len(dataloader):.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}')

# Save the trained model
#torch.save(net.state_dict(), 'trained_model.pt')

with open(os.path.join(os.getcwd(), 'models', 'NN4.pkl'), 'wb') as f:
     pickle.dump(net, f)